In [1]:
import sys
sys.path.append("..")

In [2]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms.algorithms import *
from utils.models import *
from utils.dataloaders import *

from tabulate import tabulate

In [3]:
device = 'cpu'
dataset = SyntheticDataset(device=device)

In [4]:
def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[4])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=False, 
            delta_effort=hp['delta_effort'],
            effort_iter=hp['effort_iter'],
            effort_lr=hp['effort_lr']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[4])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=False, 
            delta_effort=hp['delta_effort'],
            effort_iter=hp['effort_iter'],
            effort_lr=hp['effort_lr']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fc_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[4])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fc_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=False, 
            delta_effort=hp['delta_effort'],
            effort_iter=hp['effort_iter'],
            effort_lr=hp['effort_lr']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI_fc = lr_fc_model_runner(dataset, EI_hp_fc, seeds)
    _, _, EI_kde = lr_kde_model_runner(dataset, EI_hp_kde, seeds)
    _, _, EI_fb = lr_fb_model_runner(dataset, EI_hp_fb, seeds)
    
    return SGD, EI_fc, EI_kde, EI_fb

def fb_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fb_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

def kde_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

def fc_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fc_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

#### SGD Hyperparameter Selection

In [5]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 1024
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 0.5
SGD_hp_test['effort_iter'] = 20
SGD_hp_test['effort_lr'] = 1

kde_hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:06<00:00, 14.40epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:06<00:00, 14.47epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:06<00:00, 14.47epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:06<00:00, 14.43epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp          eo      eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ----------  --------
         0.0001          0          0.733359        0.735     0.00676745  0.0047987  0.00469479  0.130042
         0.001           0          0.792578        0.789687  0.0661657   0.105618   0.141167    0.141167
         0.01            0          0.800391        0.794687  0.153806    0.176061   0.22823     0.22823
         0.1             0          0.802266        0.795937  0.11572     0.155567   0.192647    0.192647


The learning rate is decided as 0.1.


In [6]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.1
SGD_hp['lambda_'] = 0


#### EI Hyperparameter Test

In [7]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.1]
EI_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.85, 0.9]
EI_hp_test['fairness'] = 'EI'

kde_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  7.77epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  7.79epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  7.75epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.23epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ---------  ---------
            0.1       0             0.802266        0.795937  0.11572     0.155567   0.192647   0.192647
            0.1       0.2           0.802109        0.792188  0.0909701   0.146543   0.184858   0.184858
            0.1       0.6           0.797344        0.79      0.0243368   0.0667511  0.0808395  0.0878023
            0.1       0.8           0.788594        0.780312  0.0158162   0.0536785  0.070886   0.10462
            0.1       0.85          0.78625         0.78      0.00914228  0.0276487  0.0346052  0.114994
            0.1       0.9           0.781563        0.773125  0.0213494   0.0158129  0.0323951  0.136307


The lambda for EI is decided as 0.85.

In [8]:
EI_hp_kde = EI_hp_test.copy()
EI_hp_kde['learning_rate'] = 0.1
EI_hp_kde['lambda_'] = 0.85

In [9]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.1]
EI_hp_test['lambda_'] = [0, 0.6, 0.8, 0.9, 0.95, 0.99]
EI_hp_test['fairness'] = 'EI'

fc_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.43epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  8.12epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  8.16epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  7.71epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ---------  ---------
            0.1       0             0.802266        0.795937  0.11572    0.155567   0.192647   0.192647
            0.1       0.6           0.804844        0.7975    0.0861538  0.141752   0.16869    0.16869
            0.1       0.8           0.802266        0.793125  0.0811292  0.109707   0.128655   0.128655
            0.1       0.9           0.800312        0.792188  0.0516672  0.0861135  0.100285   0.100285
            0.1       0.95          0.7975          0.78625   0.0375933  0.0737696  0.0873038  0.0873038
            0.1       0.99          0.768047        0.762188  0.0226689  0.0203053  0.0446424  0.134016


In [10]:
EI_hp_fc = EI_hp_test.copy()
EI_hp_fc['learning_rate'] = 0.1
EI_hp_fc['lambda_'] = 0.99

In [11]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.1]
EI_hp_test['lambda_'] = [0, 0.1, 0.2, 0.25, 0.4]
EI_hp_test['fairness'] = 'EI'

fb_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.52epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  7.85epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  8.07epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  8.06epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei          dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ----------  ---------  ---------
            0.1       0             0.802266        0.795937  0.11572    0.155567    0.192647   0.192647
            0.1       0.1           0.79625         0.786563  0.0325501  0.064785    0.0824063  0.0921061
            0.1       0.2           0.787734        0.773438  0.0195628  0.0380138   0.0635508  0.121009
            0.1       0.25          0.782734        0.770312  0.0208465  0.00563046  0.0185809  0.142607
            0.1       0.4           0.751797        0.742812  0.0370143  0.0360892   0.0284331  0.165408


In [12]:
EI_hp_fb = EI_hp_test.copy()
EI_hp_fb['learning_rate'] = 0.1
EI_hp_fb['lambda_'] = 0.25

#### Model training

In [13]:
seeds = np.arange(1,6)

SGD, EI_fc, EI_kde, EI_fb = experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, seeds)

/var/folders/mt/mgpv1mld71dfr_kn_fq57str0000gn/T/ipykernel_78437/4096305693.py:29: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seeds[i])


training seed 1 started


Training: 100%|██████████| 100/100 [00:06<00:00, 14.35epochs/s]


training seed 2 started


Training: 100%|██████████| 100/100 [00:06<00:00, 14.36epochs/s]


...

training seed 4 started


Training: 100%|██████████| 100/100 [00:12<00:00,  7.94epochs/s]


training seed 5 started


Training: 100%|██████████| 100/100 [00:12<00:00,  8.06epochs/s]

Training finished for all seeds.


In [14]:
result = []
models = ["SGD", "EI FC", "EI KDE", "EI FB"]
sol = [SGD, EI_fc, EI_kde, EI_fb]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model      accuracy_mean    accuracy_var    ei_mean      ei_var    dp_mean      dp_var    eo_mean     eo_var    eodd_mean    eodd_var
-------  ---------------  --------------  ---------  ----------  ---------  ----------  ---------  ---------  -----------  ----------
SGD              0.79495      0.00302242  0.14103    0.036398    0.162037   0.0220035   0.220476   0.0326993     0.220476  0.0326993
EI FC            0.7583       0.00639805  0.0043636  0.00263378  0.0078031  0.00797339  0.0280908  0.0167435     0.12906   0.00199039
EI KDE           0.7727       0.00709119  0.011013   0.0064985   0.0153957  0.00841219  0.0418284  0.0176709     0.131901  0.0128807
EI FB            0.7712       0.012042    0.0188428  0.00946359  0.0297906  0.0212023   0.0541524  0.0369255     0.117372  0.0129878
